In [ ]:
!pip uninstall -y eonacs && pip install  git+https://github.com/njnmco/smc #--log /dev/stderr
# https://stackoverflow.com/questions/67798070/raytune-is-throwing-error-module-pickle-has-no-attribute-picklebuffer-whe
# downgrade pickle5 for ray
!pip install datasets ray[tune] pickle5==0.0.10
# NB restart kernel after setup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/njnmco/smc to /tmp/pip-req-build-z_p5awh_
  Running command git clone -q https://github.com/njnmco/smc /tmp/pip-req-build-z_p5awh_
     |████████████████████████████████| 4.2 MB 27.2 MB/s 
     |████████████████████████████████| 646 kB 58.3 MB/s 
     |████████████████████████████████| 6.6 MB 59.7 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
  Created wheel for eonacs: filename=eonacs-1.0.0-py3-none-any.whl size=5532 sha256=2b0e391a3182ec9a7538b74131f90c9aceccf89a3cac55b0cf47a0b51693b995
  Stored in directory: /tmp/pip-ephem-wheel-cache-klvdfjvh/wheels/6e/02/6d/470a51584ecf5233c1da27aab5d5e137df8b9ab9cc345b2a49
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391567 sha256=89e7d7659f0bfaa8e43d762479bb3121355b0e4ed462d0a101ed61e6a355291e
  Stored in direc

In [ ]:
NUM_TRIALS = 30
OUTPUT = "/content/drive/MyDrive/smc/models/dbert_tasks_hyper_refactor"

In [ ]:
import eonacs.common.util as util

util.colab_map_drive()


Mounted at /content/drive


In [ ]:
con = util.onet_db()

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
tasks = pd.read_sql("select distinct task as text from task_statements",   con)
#dwa = pd.read_sql("select * from dwa_reference",   con)

In [ ]:
import eonacs.common.dbert as dbert
tokenizer, model = dbert.dbert()

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Fine Tuning

In [ ]:
from transformers import AutoModel, AutoModelForMaskedLM

In [ ]:
from datasets import Dataset

tasks_d = Dataset.from_pandas(tasks)

In [ ]:
tasks_d_split = tasks_d.train_test_split()

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = tasks_d_split.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 13482
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4494
    })
})

In [ ]:
chunk_size = 128


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2092
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 694
    })
})

In [ ]:
# spot check
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'related to solar installations. [SEP] [CLS] refer patients to ophthalmic surgeons or other physicians. [SEP] [CLS] gather personnel records from other departments or employees. [SEP] [CLS] develop or approve project plans, schedules, or budgets. [SEP] [CLS] set up or operate glue machines by filling glue reservoirs, turning switches to activate heating elements, or adjusting glue flow or conveyor speed. [SEP] [CLS] devise and participate in activities to improve fish hatching and growth rates, and to prevent disease in hatcheries. [SEP] [CLS] create customized energy management packages to satisfy customer needs. [SEP] [CLS] verify all financial, physical, and human resources assigned to research or development projects are used'

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)


In [ ]:
from transformers import TrainingArguments

batch_size = 32
# Show the training loss with every epoch
logging_steps = len(lm_datasets["train"]) // batch_size
model_name = "distilbert-base-uncased"

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-tasks",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    fp16=False,
    logging_steps=logging_steps,
    #below will save checkpoint of each run, so that best model can be retrieved directly w/o retraining
    save_strategy="epoch",
    save_total_limit=1
)

## Hyperparameter search

https://huggingface.co/blog/ray-tune

In [ ]:
training_args.disable_tqdm = True

In [ ]:
from transformers import Trainer
trainer = Trainer(
    args = training_args,
    train_dataset = lm_datasets["train"],
    eval_dataset = lm_datasets["test"],
    data_collator = data_collator,
    model_init = lambda : AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased', return_dict=True),
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

In [ ]:
best = trainer.hyperparameter_search(
    direction="minimize", 
    backend="ray", 
    n_trials=NUM_TRIALS,
    resume = 'AUTO'
)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Visualize fits
import tensorboard
%load_ext tensorboard
%tensorboard --logdir /root/ray_results/
#!kill 4166

Reusing TensorBoard on port 6006 (pid 3334), started 2:36:42 ago. (Use '!kill 3334' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
# reload best model from checkpoint
print(best)
path = ! find /root/ray_results/ -type d -path *run-{best.run_id}/checkpoint*
best_model = AutoModelForMaskedLM.from_pretrained(path[0])
best_model.save_pretrained(OUTPUT)

BestRun(run_id='aee76_00028', objective=2.211339235305786, hyperparameters={'learning_rate': 6.53330522022775e-05, 'num_train_epochs': 5, 'seed': 22.053802560765252, 'per_device_train_batch_size': 64})


loading configuration file /root/ray_results/_objective_2022-05-27_17-23-11/_objective_aee76_00028_28_learning_rate=6.5333e-05,num_train_epochs=5,per_device_train_batch_size=64,seed=22.054_2022-05-27_18-26-06/distilbert-base-uncased-finetuned-tasks/run-aee76_00028/checkpoint-330/config.json
Model config DistilBertConfig {
  "_name_or_path": "/root/ray_results/_objective_2022-05-27_17-23-11/_objective_aee76_00028_28_learning_rate=6.5333e-05,num_train_epochs=5,per_device_train_batch_size=64,seed=22.054_2022-05-27_18-26-06/distilbert-base-uncased-finetuned-tasks/run-aee76_00028/checkpoint-330",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_we

### Check
Verify that fine-tuned is more spread out than base model and that reloading it does not change predictions

In [ ]:
dbert.get_embeddings(tasks.loc[1:5, "text"], tokenizer, model)

Using cuda:0
0...


array([[-0.14429109,  0.09607384, -0.4877717 , ..., -0.15828916,
         0.24980731,  0.636069  ],
       [-0.22339639, -0.22542542, -0.8392943 , ..., -0.1708761 ,
         0.2203352 ,  0.33022025],
       [-0.11400064,  0.01582951, -0.40959972, ..., -0.33057234,
        -0.05825482,  0.3881769 ],
       [-0.05106229, -0.22800283, -0.50335205, ..., -0.20114018,
        -0.21932895,  0.5329145 ],
       [-0.21640758, -0.18856755, -0.37812456, ..., -0.1840276 ,
         0.3614779 ,  0.562684  ]], dtype=float32)

In [ ]:
dbert.get_embeddings(tasks.loc[1:5, "text"], tokenizer, best_model)

Using cuda:0
0...


array([[ -8.624067 ,  -8.410876 ,  -8.1354885, ...,  -7.987005 ,
         -7.701396 ,  -6.5213933],
       [ -8.982267 ,  -8.783544 ,  -8.521572 , ...,  -8.638495 ,
         -8.4410305,  -7.745404 ],
       [ -8.613484 ,  -8.493114 ,  -8.205366 , ...,  -8.344113 ,
         -7.83003  ,  -7.5580297],
       [-11.037829 , -10.704328 , -10.443352 , ...,  -9.839433 ,
         -9.848788 ,  -9.746413 ],
       [ -7.335724 ,  -7.2203865,  -7.057119 , ...,  -6.786019 ,
         -6.5046015,  -4.984431 ]], dtype=float32)

In [ ]:
reload = AutoModel.from_pretrained(OUTPUT)
dbert.get_embeddings(tasks.loc[1:5, "text"], tokenizer, reload).shape

Some weights of the model checkpoint at /content/drive/MyDrive/smc/models/dbert_tasks_hyper_refactor were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using cuda:0
0...


(5, 768)

## Save out embeddings

In [ ]:
hyper_e = dbert.get_embeddings(tasks.loc[:, "text"], tokenizer, reload)

Using cuda:0
0...
1024...
2048...
3072...
4096...
5120...
6144...
7168...
8192...
9216...
10240...
11264...
12288...
13312...
14336...
15360...
16384...
17408...


In [ ]:
hyper_e.shape

(17976, 768)

In [ ]:
pd.to_pickle((tasks, hyper_e), "/content/drive/MyDrive/smc/data/tasks_dbert_hyper_e_refactor.pkl.gz")